In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://storage.googleapis.com/gmtempor/reco_sample_dataset.csv')

In [3]:
df.head()

,partition_date,orderparentid,user_id,productcontentid,brand_id,category_id,category_name,gender,price,color_id,business_unit
0,2020-08-20 06:00:00,335057357,86386,39328996,919155,418,Sandalet,Kadın,319.98,14.0,Branded Shoes B
1,2020-08-24 10:00:00,337401625,59469,31903343,121,1827,Banyo Dolabı,Unisex,1195.56,3.0,Bahçe & Yapı Market & Hırdavat
2,2020-08-26 19:00:00,338457012,51248,34726400,40,604,T-Shirt,Kadın,37.99,14.0,PL Woman
3,2020-08-22 11:00:00,336681542,29380,32920640,7651,604,T-Shirt,Kadın,24.99,6.0,Kadın A
4,2020-08-20 19:00:00,335736916,68368,39035716,3395,599,Kazak,Kadın,79.90,16.0,Kadın A


In [4]:
from sklearn.cluster import KMeans

km = KMeans() # Gaussian Mixture is soft margin version of this.

In [15]:
km.cluster_centers_

array([[   45.55022127],
       [ 4372.26811448],
       [ 2783.74736682],
       [ 1370.2755743 ],
       [12793.08555556],
       [  162.87288865],
       [ 7609.7790411 ],
       [  471.1108517 ]])

In [5]:
df['price_cluster'] = km.fit_predict(df.price.values.reshape(-1,1))

In [6]:
import sqlite3

conn = sqlite3.connect(":memory:")

In [7]:
df[['user_id', 'price_cluster']].to_sql("price_cluster", conn)

In [8]:
user_price_profile = pd.read_sql("""

select user_id
, sum(case when price_cluster = 0 then n else 0 end) as pc0 
, sum(case when price_cluster = 1 then n else 0 end) as pc1
, sum(case when price_cluster = 2 then n else 0 end) as pc2 
, sum(case when price_cluster = 3 then n else 0 end) as pc3 
, sum(case when price_cluster = 4 then n else 0 end) as pc4 
, sum(case when price_cluster = 5 then n else 0 end) as pc5 
, sum(case when price_cluster = 6 then n else 0 end) as pc6 
, sum(case when price_cluster = 7 then n else 0 end) as pc7 

from 
(select user_id, price_cluster,count(1) n 
from  price_cluster 
group by user_id, price_cluster) foo
group by user_id
""", conn)

In [9]:
df[['productcontentid','price_cluster']].to_sql("product_profile", conn)

In [10]:
product_price_profile = pd.read_sql("""

select productcontentid
, sum(case when price_cluster = 0 then n else 0 end) > 0 as pc0 
, sum(case when price_cluster = 1 then n else 0 end) >0 as pc1
, sum(case when price_cluster = 2 then n else 0 end) >0 as pc2 
, sum(case when price_cluster = 3 then n else 0 end) >0 as pc3 
, sum(case when price_cluster = 4 then n else 0 end)>0 as pc4 
, sum(case when price_cluster = 5 then n else 0 end)>0 as pc5 
, sum(case when price_cluster = 6 then n else 0 end)>0 as pc6 
, sum(case when price_cluster = 7 then n else 0 end)>0 as pc7 

from 
(select productcontentid, price_cluster,count(1) n 
from  product_profile 
group by productcontentid, price_cluster) foo
group by productcontentid
""", conn)

product_price_profile

,productcontentid,pc0,pc1,pc2,pc3,pc4,pc5,pc6,pc7
0,51971,1,0,0,0,0,0,0,0
1,51973,1,0,0,0,0,0,0,0
2,51974,1,0,0,0,0,0,0,0
3,51992,1,0,0,0,0,0,0,0
4,51994,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
184087,47005516,1,0,0,0,0,0,0,0
184088,47005525,1,0,0,0,0,0,0,0
184089,47005548,1,0,0,0,0,0,0,0
184090,47048393,1,0,0,0,0,0,0,0


In [11]:
from sklearn.neighbors import KDTree

In [12]:
kd = KDTree(product_price_profile.drop('productcontentid',axis=1).values)

In [14]:
kd.query(user_price_profile[user_price_profile.user_id == 102].drop('user_id',axis=1).values, k=50)

(array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]]),
 array([[47035, 61702, 14306, 60071, 58441, 65359, 49685, 60070, 16532,
         47033, 64232, 68419, 63974, 47150, 53039, 60976,  2951, 56796,
         47794, 59624,  3107, 14264, 54552, 54553, 16072, 55315, 61445,
          2952, 47151,  3756,  2956, 14022, 55323, 54919, 60480, 13392,
         51978, 49195, 46097, 61427,  3458, 65348, 53144, 64783, 54551,
          3783, 61438, 47724, 11996, 68417]]))

In [ ]:
user_price_profile.head()